In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split

In [4]:
# --- Caminhos dos arquivos de entrada (baseado na estrutura do repositório) ---
OASIS_CSV = '../database/oasis_longitudinal_demographic.csv'
FEATURES_CSV = '../database/features_full.csv'
IDENTIFIERS_CSV = '../database/features_identifiers.csv'

# Assumindo que elas estão em uma pasta raiz, talvez baixadas separadamente.
SOURCE_IMG_DIR = '../database/axl'

# --- Caminhos de saída ---
TREINO_DIR = '../database/treino'
TESTE_DIR = '../database/teste'
TREINO_IMG_DIR = os.path.join(TREINO_DIR, 'imgs')
TESTE_IMG_DIR = os.path.join(TESTE_DIR, 'imgs')

# --- Parâmetros da Divisão ---
TEST_SIZE = 0.20
RANDOM_STATE = 42

print(f"Diretório de treino: {TREINO_DIR}")
print(f"Diretório de teste: {TESTE_DIR}")

os.makedirs(TREINO_IMG_DIR, exist_ok=True)
os.makedirs(TESTE_IMG_DIR, exist_ok=True)

print(f"Pastas {TREINO_IMG_DIR} e {TESTE_IMG_DIR} verificadas/criadas.")

Diretório de treino: ../database/treino
Diretório de teste: ../database/teste
Pastas ../database/treino/imgs e ../database/teste/imgs verificadas/criadas.


In [9]:
# Carrega o dataframe principal
try:
    # Adicionado decimal=',' para ler corretamente os números (ex: 0,5) como floats
    df_oasis = pd.read_csv(OASIS_CSV, sep=';', decimal=',')
except FileNotFoundError:
    print(f"Erro: Arquivo {OASIS_CSV} não encontrado.")
    # Se der erro aqui, pare ou trate
    raise

print(f"Forma original df_oasis: {df_oasis.shape}")
print(f"Valores originais em 'Group': {df_oasis['Group'].unique()}")
print(f"Tipo de dado da coluna CDR (deve ser float): {df_oasis['CDR'].dtype}")

# Função para mapear 'Converted'
def map_class(row):
    if row['Group'] == 'Converted':
        if row['CDR'] > 0:
            return 'Demented'
        else: # Inclui CDR = 0 e CDR = NaN (que falha no > 0 e cai aqui)
            return 'NonDemented'
    return row['Group'] # Retorna 'Demented' ou 'NonDemented'

# Aplica a função para criar a classe final
df_oasis['Group'] = df_oasis.apply(map_class, axis=1)

# Filtra o dataframe para conter apenas as duas classes finais
df_oasis = df_oasis[df_oasis['Group'].isin(['Demented', 'NonDemented'])].copy()

print(f"Valores finais em 'Group': {df_oasis['Group'].unique()}")
print(f"Forma final df_oasis: {df_oasis.shape}")

Forma original df_oasis: (373, 15)
Valores originais em 'Group': ['Nondemented' 'Demented' 'Converted']
Tipo de dado da coluna CDR (deve ser float): float64
Valores finais em 'Group': ['Demented' 'NonDemented']
Forma final df_oasis: (183, 15)


In [10]:
# Mapeia Demented=1, NonDemented=0
df_oasis['Group_num'] = df_oasis['Group'].map({'Demented': 1, 'NonDemented': 0})

# Agrupa por paciente (Subject ID) e pega o valor máximo (1 se Demented, 0 se NonDemented)
patient_labels = df_oasis.groupby('Subject ID')['Group_num'].max()

# Remove pacientes que possam ter tido apenas visitas com NaN (se houver)
patient_labels = patient_labels.dropna()

print(f"Total de pacientes únicos para divisão: {len(patient_labels)}")
print("Contagem de pacientes por classe (para estratificação):")
print(patient_labels.value_counts())

Total de pacientes únicos para divisão: 78
Contagem de pacientes por classe (para estratificação):
Group_num
1    78
Name: count, dtype: int64


In [11]:
# Obtém a lista de pacientes (índice) e os rótulos (valores)
patient_ids = patient_labels.index
labels = patient_labels.values

# Divide os pacientes
train_patients, test_patients, _, _ = train_test_split(
    patient_ids,
    labels,
    test_size=TEST_SIZE,
    stratify=labels,
    random_state=RANDOM_STATE
)

print(f"Total de pacientes de treino: {len(train_patients)}")
print(f"Total de pacientes de teste: {len(test_patients)}")

Total de pacientes de treino: 62
Total de pacientes de teste: 16


In [12]:
try:
    # Adicionando decimal=',' aqui também por segurança, 
    # caso os arquivos de features também usem vírgula.
    df_feat = pd.read_csv(FEATURES_CSV, sep=';', decimal=',')
    df_ids = pd.read_csv(IDENTIFIERS_CSV, sep=';', decimal=',')
except FileNotFoundError:
    print(f"Erro: Arquivos {FEATURES_CSV} ou {IDENTIFIERS_CSV} não encontrados.")
    raise
except Exception as e:
    print(f"Erro ao ler CSVs de features: {e}")
    print("Verifique se o separador (sep=';') está correto para ambos os arquivos.")
    raise

# --- CORREÇÃO ---
# Verifica se 'MRI ID' (com 'I') está presente
if 'MRI ID' not in df_feat.columns or 'MRI ID' not in df_ids.columns:
    print("ALERTA: 'MRI ID' não encontrado em um dos dataframes de features.")
    print("O script pode falhar ou filtrar incorretamente.")
else:
    print("'MRI ID' encontrado nos CSVs de features.")
# --- FIM DA CORREÇÃO ---


print(f"Forma df_feat: {df_feat.shape}")
print(f"Forma df_ids: {df_ids.shape}")

def save_datasets(patient_list, output_dir_base):
    """
    Filtra todos os 3 dataframes pela lista de pacientes e salva os CSVs no diretório de saída.
    """
    print(f"\nProcessando diretório: {output_dir_base}")
    
    # 1. Filtrar df_oasis
    df_oasis_set = df_oasis[df_oasis['Subject ID'].isin(patient_list)]
    
    # --- CORREÇÃO ---
    # 2. Obter os MRI IDs (exames) únicos deste conjunto de pacientes
    mr_ids_set = df_oasis_set['MRI ID'].unique()
    # --- FIM DA CORREÇÃO ---
    
    print(f"  Pacientes: {len(patient_list)}, Exames (MRI IDs): {len(mr_ids_set)}")
    
    # --- CORREÇÃO ---
    # 3. Filtrar df_ids e df_feat usando os MRI IDs
    # Assumindo que 'MRI ID' existe em ambos, conforme verificação anterior
    df_ids_set = df_ids[df_ids['MRI ID'].isin(mr_ids_set)]
    df_feat_set = df_feat[df_feat['MRI ID'].isin(mr_ids_set)]
    # --- FIM DA CORREÇÃO ---
    
    # 4. Salvar os CSVs
    try:
        # Salva com vírgula decimal para consistência
        df_oasis_set.to_csv(os.path.join(output_dir_base, 'oasis_longitudinal_demographic.csv'), index=False, sep=';', decimal=',')
        df_ids_set.to_csv(os.path.join(output_dir_base, 'features_identifiers.csv'), index=False, sep=';', decimal=',')
        df_feat_set.to_csv(os.path.join(output_dir_base, 'features_full.csv'), index=False, sep=';', decimal=',')
        print(f"  CSVs salvos com sucesso em {output_dir_base}")
    except Exception as e:
        print(f"  Erro ao salvar CSVs em {output_dir_base}: {e}")

# Salvar conjuntos de Treino (80%)
save_datasets(train_patients, TREINO_DIR)

# Salvar conjuntos de Teste (20%)
save_datasets(test_patients, TESTE_DIR)

print("\n--- Processamento de CSVs Concluído ---")

'MRI ID' encontrado nos CSVs de features.
Forma df_feat: (373, 21)
Forma df_ids: (373, 9)

Processando diretório: ../database/treino
  Pacientes: 62, Exames (MRI IDs): 143
  CSVs salvos com sucesso em ../database/treino

Processando diretório: ../database/teste
  Pacientes: 16, Exames (MRI IDs): 40
  CSVs salvos com sucesso em ../database/teste

--- Processamento de CSVs Concluído ---


In [13]:
def copy_image_files(patient_list, destination_img_dir):
    """
    Encontra e copia os arquivos .nii/.nii.gz para o destino (treino/imgs ou teste/imgs).
    """
    print(f"\nCopiando imagens para: {destination_img_dir}")
    
    if not os.path.exists(SOURCE_IMG_DIR):
        print(f"  ALERTA: Diretório de origem das imagens não encontrado: {SOURCE_IMG_DIR}")
        print("  Nenhuma imagem será copiada. Ajuste a variável 'SOURCE_IMG_DIR'.")
        return

    # Lista de todos os arquivos de imagem de origem
    try:
        source_files = [f for f in os.listdir(SOURCE_IMG_DIR) if f.endswith(('.nii', '.nii.gz'))]
    except Exception as e:
        print(f"  Erro ao listar arquivos em {SOURCE_IMG_DIR}: {e}")
        return
        
    # --- CORREÇÃO ---
    # Pega os MRI IDs do conjunto de pacientes
    mr_ids_set = df_oasis[df_oasis['Subject ID'].isin(patient_list)]['MRI ID'].unique()
    # --- FIM DA CORREÇÃO ---
    
    copied_count = 0
    missing_count = 0
    
    # Itera sobre os MR IDs que precisamos
    for mr_id in mr_ids_set:
        found_file = None
        # Procura o arquivo de origem que contém o MR ID
        # (ex: 'OAS2_0001_MR1' deve estar no nome do arquivo)
        for fname in source_files:
            if mr_id in fname:
                found_file = fname
                break # Encontrou o arquivo para este MR ID
        
        if found_file:
            source_path = os.path.join(SOURCE_IMG_DIR, found_file)
            dest_path = os.path.join(destination_img_dir, found_file)
            
            if not os.path.exists(dest_path): # Evita copiar se já existir
                try:
                    shutil.copy(source_path, dest_path)
                    copied_count += 1
                except Exception as e:
                    print(f"  Erro ao copiar {source_path}: {e}")
            else:
                copied_count += 1 # Conta como 'copiado' se já estiver lá
        else:
            print(f"  Aviso: Imagem para MRI ID {mr_id} não encontrada em {SOURCE_IMG_DIR}")
            missing_count += 1
            
    print(f"  Copiadas/Verificadas: {copied_count} imagens.")
    print(f"  Não encontradas: {missing_count} imagens.")

# Copiar imagens de TREINO
copy_image_files(train_patients, TREINO_IMG_DIR)

# Copiar imagens de TESTE
copy_image_files(test_patients, TESTE_IMG_DIR)

print("\n--- Processamento de Imagens Concluído ---")


Copiando imagens para: ../database/treino/imgs
  Copiadas/Verificadas: 143 imagens.
  Não encontradas: 0 imagens.

Copiando imagens para: ../database/teste/imgs
  Copiadas/Verificadas: 40 imagens.
  Não encontradas: 0 imagens.

--- Processamento de Imagens Concluído ---


In [14]:
print("Divisão 80/20 (Treino/Teste) concluída.")
print("Verifique as pastas './database/treino' e './database/teste'.")

Divisão 80/20 (Treino/Teste) concluída.
Verifique as pastas './database/treino' e './database/teste'.
